In [8]:
"""
Métodos para gerar os dados do financiamento
"""
import locale
from datetime import datetime

import pandas as pd
from dateutil import rrule


def calcular_juros(valor_do_financiamento: float,
                   taxa_de_juros: float) -> float:
    """Calcula o valor de juros do financiamento
    """
    taxa_mensal_decimal = (1 + taxa_anual_decimal) ** (1 / 12) - 1


def calcular_parcela(valor_amortizacao: float,
                     valor_juros: float,
                     demais_valores: float = 0) -> float:
    """Calcula o valor da parcela do financiamento
    """
    return valor_amortizacao + valor_juros + demais_valores


def calcular_amortizacao(quantidade_parcelas: int, valor_emprestimo: float) -> float:
    """ Calcula o valor de amortização dada a quantidade de parcelas e valor do financiamento
    """
    valor_amortizacao = valor_emprestimo / quantidade_parcelas
    return valor_amortizacao


def gerar_dados(valor_financiamento: float,
                taxa_juros: float,
                quantidade_parcelas: int,
                amortizacao: float = 0) -> list[tuple]:
    """Gera/calcula os dadas do financiamento
    """
    amortizacao_financiamento = calcular_amortizacao(quantidade_parcelas, valor_financiamento)
    dados = []
    amortizacao_adicional = amortizacao

    for i in range(quantidade_parcelas):

        juros = calcular_juros(valor_financiamento, taxa_juros)
        amortizacao_total = amortizacao_financiamento + amortizacao_adicional
        valor_parcela = calcular_parcela(amortizacao_financiamento, juros)
        valor_financiamento -= amortizacao_total

        if valor_financiamento - amortizacao_financiamento < 0:
            amortizacao_adicional += valor_financiamento
            valor_financiamento -= valor_financiamento
            adicionar_dados(
                valor_parcela,
                amortizacao_financiamento,
                amortizacao_adicional,
                juros,
                valor_financiamento,
                dados
            )

            break

        adicionar_dados(
            valor_parcela,
            amortizacao_financiamento,
            amortizacao_adicional,
            juros,
            valor_financiamento,
            dados)
    # TODO: Adicionar total em um dataframe separado e concatatenar tendo um index 'total'

    return dados


def calcular_totais(dados: list[tuple]) -> tuple:
    """Calcula o total pago individualmente nas parcelas, juros e amortizações.
"""

    valor_parcela, amortizacao_financiamento, amortizacao_adicional, juros = 0, 0, 0, 0

    for each in dados:
        valor_parcela += each[0]
        amortizacao_financiamento += each[1]
        amortizacao_adicional += each[2]
        juros += each[3]

    return valor_parcela, amortizacao_financiamento, amortizacao_adicional, juros


def adicionar_dados(valor_parcela: float,
                    amortizacao_financiamento: float,
                    amortizacao_adicional: float,
                    juros: float,
                    valor_financiamento: float,
                    dados_financiamento: list[tuple]) -> None:
    """ Adiciona em uma lista todos os dados necessarios para montar a tabela com a simulação do financiamento
    """
    dados_financiamento.append(
        (valor_parcela,
         amortizacao_financiamento,
         amortizacao_adicional,
         juros,
         valor_financiamento))


def gerar_tabela_meses(dados_calculados: list[tuple],
                       data_inicio: str = datetime.strftime(datetime.now(), "%m/%y")) -> str:
    """Gerar uma representação em forma de tabela a partir de uma lista de dados, onde o index informa os meses de pagamento do financiamento.

    >>> print(gerar_tabela_meses(list(f for f in gerar_dados(10000, 7.4, 2))))
            Parcela Amortização Amortização adicional  Juros Saldo Devedor
    07/22  5.061,67    5.000,00                  0,00  61,67      5.000,00
    08/22  5.030,83    5.000,00                  0,00  30,83          0,00
    """
    dados_formatados = []
    for i in dados_calculados:
        dados_formatados.append([formatar_valor(j) for j in i])

    index = list(f for f in configurar_meses(dados_formatados, data_inicio))
    df = pd.DataFrame(dados_formatados,
                      columns=['Parcela', 'Amortização', 'Amortização adicional', 'Juros', 'Saldo Devedor'],
                      index=index)
    return df


def gerar_tabela_parcela(dados_calculados: list[tuple]) -> str:
    """Gerar uma representação em forma de tabela a partir de uma lista de dados, onde o index informa o número de parcelas do financiamento.

    :param dados_calculados: lista com os dados das parcelas do financiamento.
    :return: Conteudo do dataframe como string

    **Examples**

    >>> print(gerar_tabela_parcela(list(f for f in gerar_dados(10000, 7.4, 2))))
        Parcela Amortização Amortização adicional  Juros Saldo Devedor
    1  5.061,67    5.000,00                  0,00  61,67      5.000,00
    2  5.030,83    5.000,00                  0,00  30,83          0,00
    """

    dados_formatados = []

    for i in dados_calculados:
        dados_formatados.append([formatar_valor(j) for j in i])

    df = pd.DataFrame(dados_formatados,
                      columns=[
                          'Parcela', 'Amortização',
                          'Amortização adicional',
                          'Juros', 'Saldo Devedor'
                      ],
                      index=pd.RangeIndex(start=1, stop=len(dados_formatados) + 1))

    return df


def configurar_meses(dados: list[tuple],
                     data_inicio: str = datetime.strftime(datetime.now(), "%m/%y")
                     ) -> list[str]:
    """Criar lista com range de datas formatadas
    >>> configurar_meses(list(e for e in range(2)), '07/22')
    ['07/22', '08/22']
    """
    start_strptime = datetime.strptime(data_inicio, "%m/%y")
    return [e.strftime("%m/%y") for e in
            list(rrule.rrule(rrule.MONTHLY, count=len(dados), dtstart=start_strptime))]


def formatar_valor(valor: float) -> str:
    """Formatar os valores para o formato brasileiro (000.000,00)

    **Examples**

    >>> formatar_valor(1000.1245)
    '1.000,12'
    """
    locale.setlocale(locale.LC_ALL, 'pt_BR')
    return locale.format_string("%.2f", valor, grouping=True)


def criar_arquivo(conteudo: str) -> None:
    with open("simulacao_financiamento.html", "w") as f:
        f.writelines(conteudo)

In [ ]:
# Dados do financiamento
valor_do_financiamento = 500000
taxa_de_juros = 13.25
quantidade_parcelas = 48

# Funções

# Gerar os dados do financiamento
dados_financiamento = gerar_dados(valor_do_financiamento, taxa_de_juros, quantidade_parcelas)

# Gerar a tabela de parcelas
tabela_parcelas = gerar_tabela_parcela(dados_financiamento)

# Exibir a tabela de parcelas
print(tabela_parcelas)

# Gerar a tabela de meses
tabela_meses = gerar_tabela_meses(dados_financiamento)

# Exibir a tabela de meses
print(tabela_meses)

# Calcular totais
totais = calcular_totais(dados_financiamento)
print(f"Total pago: {formatar_valor(totais[0])}")
print(f"Total amortizado: {formatar_valor(totais[1])}")
print(f"Total amortização adicional: {formatar_valor(totais[2])}")
print(f"Total juros: {formatar_valor(totais[3])}")

# Criar arquivo HTML com a tabela de meses
criar_arquivo(tabela_meses.to_html())


      Parcela Amortização Amortização adicional     Juros Saldo Devedor
1   15.628,12   10.416,67                  0,00  5.211,45    489.583,33
2   15.519,55   10.416,67                  0,00  5.102,88    479.166,67
3   15.410,98   10.416,67                  0,00  4.994,31    468.750,00
4   15.302,41   10.416,67                  0,00  4.885,74    458.333,33
5   15.193,83   10.416,67                  0,00  4.777,17    447.916,67
6   15.085,26   10.416,67                  0,00  4.668,59    437.500,00
7   14.976,69   10.416,67                  0,00  4.560,02    427.083,33
8   14.868,12   10.416,67                  0,00  4.451,45    416.666,67
9   14.759,55   10.416,67                  0,00  4.342,88    406.250,00
10  14.650,97   10.416,67                  0,00  4.234,31    395.833,33
11  14.542,40   10.416,67                  0,00  4.125,73    385.416,67
12  14.433,83   10.416,67                  0,00  4.017,16    375.000,00
13  14.325,26   10.416,67                  0,00  3.908,59    364

In [10]:
# Gerar os dados do financiamento
dados_financiamento = gerar_dados(valor_do_financiamento, taxa_de_juros, quantidade_parcelas)

# Acessar a 32ª parcela (índice 31 na lista)
parcela_32 = dados_financiamento[31]

# Formatar os valores da 32ª parcela
parcela_formatada = [formatar_valor(valor) for valor in parcela_32]

# Exibir os detalhes da 32ª parcela
print(f"Detalhes da 32ª parcela:")
print(f"Parcela: {parcela_formatada[0]}")
print(f"Amortização: {parcela_formatada[1]}")
print(f"Amortização adicional: {parcela_formatada[2]}")
print(f"Juros: {parcela_formatada[3]}")
print(f"Saldo Devedor: {parcela_formatada[4]}")

Detalhes da 32ª parcela:
Parcela: 12.262,39
Amortização: 10.416,67
Amortização adicional: 0,00
Juros: 1.845,72
Saldo Devedor: 166.666,67
